In [1]:
import numpy as np

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Load dataset
crop = pd.read_csv('Crop_recommendation.csv')

In [4]:
# Map crop labels to numerical values
crop_dict = {
    'rice': 1, 'maize': 2, 'jute': 3, 'cotton': 4, 'coconut': 5, 'papaya': 6,
    'orange': 7, 'apple': 8, 'muskmelon': 9, 'watermelon': 10, 'grapes': 11,
    'mango': 12, 'banana': 13, 'pomegranate': 14, 'lentil': 15, 'blackgram': 16,
    'mungbean': 17, 'mothbeans': 18, 'pigeonpeas': 19, 'kidneybeans': 20,
    'chickpea': 21, 'coffee': 22
}
crop['label_num'] = crop['label'].map(crop_dict)

# Split data into features and labels
X = crop.iloc[:, :-2]
y = crop['label_num']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train classifiers
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Define profit dictionary
profit_dict = {
    'rice': 5000, 'maize': 3000, 'jute': 2000, 'cotton': 3500, 'coconut': 4000,
    'papaya': 2500, 'orange': 2800, 'apple': 6000, 'muskmelon': 1800,
    'watermelon': 1500, 'grapes': 5500, 'mango': 7000, 'banana': 4500,
    'pomegranate': 5000, 'lentil': 2200, 'blackgram': 2000, 'mungbean': 1500,
    'mothbeans': 1300, 'pigeonpeas': 2200, 'kidneybeans': 2300, 'chickpea': 2500,
    'coffee': 9000
}

In [12]:
#  Function for multiple crop prediction with probabilities
def predict_crops(input_data):
    
    prediction = knn_model.predict(input_data)
    probabilities = knn_model.predict_proba(input_data)

    top_predictions = []
    for prob in probabilities:
        top2_indices = prob.argsort()[-4:][::-1]
        prediction_for_input = []
        for i in top2_indices:
            crop_name = list(crop_dict.keys())[list(crop_dict.values()).index(i + 1)]
            prediction_for_input.append((crop_name, prob[i]))
        top_predictions.append(prediction_for_input)

    return top_predictions


# Function for land allocation and profit prediction
def recommend_land_allocation(input_data, total_land_area):
    top_crops = predict_crops(input_data)
    total_profit = {crop: profit_dict.get(crop, 0) for crop, _ in top_crops[0]}

    sorted_crops = sorted(total_profit.items(), key=lambda x: x[1], reverse=True)
    land_allocations = {}
    remaining_area = total_land_area
    for crop, profit in sorted_crops:
        allocated_land = remaining_area * (profit / sum([profit for _, profit in sorted_crops]))
        land_allocations[crop] = allocated_land
        remaining_area -= allocated_land

    return land_allocations

# Function to predict fertilizer
fertilizer_dict = {
    'rice': 'Urea', 'maize': 'DAP', 'jute': 'Potash', 'cotton': 'Super Phosphate',
    'coconut': 'Ammonium Sulfate', 'papaya': 'Nitrogen Fertilizer',
    'orange': 'NPK', 'apple': 'Calcium Nitrate', 'muskmelon': 'Zinc Sulfate',
    'watermelon': 'Potash', 'grapes': 'Magnesium Sulfate', 'mango': 'Bio-fertilizers',
    'banana': 'Organic Manure', 'pomegranate': 'DAP', 'lentil': 'Nitrogen Fertilizer',
    'blackgram': 'Potash', 'mungbean': 'Urea', 'mothbeans': 'NPK',
    'pigeonpeas': 'Phosphate Fertilizer', 'kidneybeans': 'Organic Manure',
    'chickpea': 'Nitrogen Fertilizer', 'coffee': 'Ammonium Nitrate'
}

In [16]:
# Example usage
input_data = pd.DataFrame({
    'N': [90], 'P': [42], 'K': [43], 'temperature': [23], 'humidity': [85],
    'pH': [6.5], 'rainfall': [200]
})

total_land_area = 10

# Predict crops
top_crops = predict_crops(input_data)
print("Top crop predictions with probabilities:", top_crops)

# Recommend land allocation
land_allocations = recommend_land_allocation(input_data, total_land_area)
print("Land allocation recommendations (in acres):", land_allocations)

# Recommend fertilizer for the top crop
top_crop = top_crops[0][0][0]
def recommend_fertilizer(crop_name):
    return fertilizer_dict.get(crop_name, "No specific recommendation")
fertilizer = recommend_fertilizer(top_crop)
print(f"Recommended fertilizer for {top_crop}: {fertilizer}")



Top crop predictions with probabilities: [[('rice', np.float64(1.0)), ('coffee', np.float64(0.0)), ('kidneybeans', np.float64(0.0)), ('pigeonpeas', np.float64(0.0))]]
Land allocation recommendations (in acres): {'coffee': 4.864864864864865, 'rice': 1.3878743608473338, 'kidneybeans': 0.4658756638303753, 'pigeonpeas': 0.3902187698922345}
Recommended fertilizer for rice: Urea


c:\Users\varun\Downloads\mds2\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\varun\Downloads\mds2\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\varun\Downloads\mds2\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
c:\Users\varun\Downloads\mds2\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
